# Teil 8 - Einführung in Pläne


### Kontext

Wir stellen hier ein Objekt vor, das für die Skalierung auf industrielles föderiertes Lernen von entscheidender Bedeutung ist: den Plan. Es reduziert die Bandbreitennutzung drastisch, ermöglicht asynchrone Schemata und gibt Remote-Geräten mehr Autonomie. Das ursprüngliche Konzept des Plans ist in der Veröffentlichung [Auf dem Weg zu föderiertem Lernen im Maßstab: Systemdesign] (https://arxiv.org/pdf/1902.01046.pdf) zu finden, wurde jedoch in der PySyft-Bibliothek an unsere Anforderungen angepasst.

Ein Plan soll genau wie eine Funktion eine Folge von Brenneroperationen speichern, ermöglicht es jedoch, diese Folge von Operationen an entfernte Mitarbeiter zu senden und einen Verweis darauf zu behalten. Auf diese Weise müssen Sie jetzt eine einzelne Nachricht mit den Referenzen des Plans und der Zeiger senden, um diese Folge von $ n $ -Operationen für einige Remote-Eingaben, auf die über Zeiger verwiesen wird, aus der Ferne zu berechnen, anstatt $ n $ -Nachrichten zu senden. Sie können Tensoren auch Ihre Funktion (die wir _state tensors_ nennen) zur Verfügung stellen, um erweiterte Funktionen zu haben. Pläne können entweder als eine Funktion angezeigt werden, die Sie senden können, oder als eine Klasse, die auch remote gesendet und ausgeführt werden kann. Daher verschwindet für Benutzer auf hoher Ebene der Begriff des Plans und wird durch eine magische Funktion ersetzt, die es ermöglicht, beliebige Funktionen, die sequentielle Brennerfunktionen enthalten, an entfernte Mitarbeiter zu senden.

Zu beachten ist, dass die Klasse von Funktionen, die Sie in Pläne umwandeln können, derzeit ausschließlich auf Sequenzen von Fackeloperationen mit Haken beschränkt ist. Dies schließt insbesondere logische Strukturen wie "if" -, "for" - und "while" -Anweisungen aus, selbst wenn wir daran arbeiten, bald Problemumgehungen zu finden. _Um genau zu sein, können Sie diese verwenden, aber der logische Pfad, den Sie bei der ersten Berechnung Ihres Plans einschlagen (zuerst "wenn" zu "Falsch" und 5 Schleifen in "für"), wird für alle nächsten Berechnungen beibehalten. was wir in den meisten Fällen vermeiden wollen.

Autoren:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - GitHub: [@LaRiffle](https://github.com/LaRiffle)
- Bobby Wagner - Twitter [@bobbyawagner](https://twitter.com/bobbyawagner) - GitHub: [@ robert-wagner](https://github.com/robert-wagner)
- Marianne Monteiro - Twitter [@hereismari](https://twitter.com/hereismari) - GitHub: [@ mari-linhares](https://github.com/mari-linhares)

Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

### Importe und Modellspezifikationen

Lassen Sie uns zuerst die offiziellen Importe machen.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Und als die für PySyft spezifischen, mit einem wichtigen Hinweis: ** Der lokale Mitarbeiter sollte kein Client-Mitarbeiter sein. ** * Nicht-Client-Mitarbeiter können Objekte speichern, und wir benötigen diese Fähigkeit, um einen Plan auszuführen. *

In [ ]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

Wir definieren Remote Worker oder Geräte, um mit den im Referenzartikel angegebenen Begriffen übereinzustimmen.
Wir stellen ihnen einige Daten zur Verfügung.

In [ ]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

### Grundlegendes Beispiel

Definieren wir eine Funktion, die wir in einen Plan umwandeln möchten. Dazu müssen Sie lediglich einen Dekorator über der Funktionsdefinition hinzufügen!

In [ ]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

Lassen Sie uns überprüfen, ja, wir haben jetzt einen Plan!

In [ ]:
plan_double_abs

Um einen Plan zu verwenden, benötigen Sie zwei Dinge: den Plan zu erstellen (dh die in der Funktion vorhandene Abfolge von Operationen zu registrieren) und ihn an einen Arbeiter / ein Gerät zu senden. Zum Glück können Sie dies sehr einfach tun!

#### Plan erstellen

Um einen Plan zu erstellen, müssen Sie ihn nur für einige Daten aufrufen.

Lassen Sie uns zunächst einen Verweis auf einige entfernte Daten erhalten: Eine Anforderung wird über das Netzwerk gesendet und ein Referenzzeiger wird zurückgegeben.

In [ ]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

Wenn wir dem Plan mitteilen, dass er remote auf dem Geräteort ausgeführt werden `location:device_1` ... wird eine Fehlermeldung angezeigt, da der Plan noch nicht erstellt wurde.

In [ ]:
plan_double_abs.is_built

In [ ]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

Um einen Plan zu erstellen, müssen Sie nur `build` für den Plan aufrufen und die zur Ausführung des Plans erforderlichen Argumente übergeben (a.k.a. einige Daten). Wenn ein Plan erstellt wird, werden alle Befehle nacheinander vom lokalen Worker ausgeführt, vom Plan abgefangen und in seinem Attribut `readable_plan` gespeichert!

In [ ]:
plan_double_abs.build(torch.tensor([1., -2.]))

In [ ]:
plan_double_abs.is_built

Wenn wir versuchen, den Plan jetzt zu senden, funktioniert es!

In [ ]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

Wie bei den Tensoren erhalten wir einen Zeiger auf das gesendete Objekt. Hier wird es einfach als `PointerPlan` bezeichnet.

Eine wichtige Sache, an die Sie sich erinnern sollten, ist, dass wir beim Erstellen eines Plans vor der Berechnung die ID (s) voreingestellt haben, unter denen die Ergebnisse gespeichert werden sollen. Auf diese Weise können Befehle asynchron gesendet werden, um bereits einen Verweis auf ein virtuelles Ergebnis zu haben und lokale Berechnungen fortzusetzen, ohne auf die Berechnung des Remote-Ergebnisses zu warten. Eine wichtige Anwendung ist, wenn Sie die Berechnung eines Stapels auf Gerät_1 benötigen und nicht warten möchten, bis diese Berechnung beendet ist, um eine weitere Stapelberechnung auf Gerät_2 zu starten.

#### Remote-Ausführung eines Plans

Wir können den Plan jetzt remote ausführen, indem wir den Zeiger auf den Plan mit einem Zeiger auf einige Daten aufrufen. Dadurch wird ein Befehl zum Remote-Ausführen dieses Plans ausgegeben, sodass der vordefinierte Speicherort der Ausgabe des Plans jetzt das Ergebnis enthält (denken Sie daran, dass wir den Speicherort des Ergebnisses vor der Berechnung voreingestellt haben). Dies erfordert auch eine einzige Kommunikationsrunde.

Das Ergebnis ist einfach ein Zeiger, genau wie wenn Sie eine übliche Hakenfackelfunktion aufrufen!

In [ ]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

In [ ]:
Und Sie können den Wert einfach zurückfordern.

In [ ]:
pointer_to_result.get()

### Auf dem Weg zu einem konkreten Beispiel

Aber wir wollen Plan auf tiefes und föderiertes Lernen anwenden, oder? Schauen wir uns also ein etwas komplizierteres Beispiel an, bei dem neuronale Netze verwendet werden, da Sie möglicherweise bereit sind, sie zu verwenden.
Beachten Sie, dass wir jetzt eine Klasse in einen Plan `umwandeln. Dazu erben wir unsere Klasse von sy.Plan (anstatt von nn.Module zu erben).

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)



In [ ]:
net=Net()

In [ ]:
net

Lassen Sie uns den Plan anhand einiger Scheindaten erstellen.

In [ ]:
net.build(torch.tensor([1., 2.]))

Wir senden den Plan jetzt an einen Remote-Mitarbeiter

In [ ]:
pointer_to_net = net.send(device_1)
pointer_to_net

Lassen Sie uns einige entfernte Daten abrufen

In [ ]:
pointer_to_data = device_1.search('input_data')[0]

Dann ist die Syntax genau wie bei der normalen sequentiellen Remote-Ausführung, dh genau wie bei der lokalen Ausführung. Im Vergleich zur klassischen Remote-Ausführung gibt es jedoch für jede Ausführung eine einzige Kommunikationsrunde.

In [ ]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

Und wir bekommen das Ergebnis wie gewohnt!

In [ ]:
pointer_to_result.get()

Et voilà! Wir haben gesehen, wie die Kommunikation zwischen dem lokalen Mitarbeiter (oder Server) und den Remote-Geräten drastisch reduziert werden kann!

### Zwischen Arbeitern wechseln

Ein wichtiges Merkmal, das wir haben möchten, ist die Verwendung des gleichen Plans für mehrere Mitarbeiter, den wir abhängig von der von uns in Betracht gezogenen entfernten Datenmenge ändern würden.
Insbesondere möchten wir den Plan nicht jedes Mal neu erstellen, wenn wir den Arbeitnehmer wechseln. Lassen Sie uns anhand des vorherigen Beispiels mit unserem kleinen Netzwerk sehen, wie wir dies tun.

In [ ]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [ ]:
net = Net()

# Build plan
net.build(torch.tensor([1., 2.]))

Hier sind die wichtigsten Schritte, die wir gerade ausgeführt haben

In [ ]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

Tatsächlich können Sie andere PointerPlans aus demselben Plan erstellen, sodass die Syntax für die Remoteausführung eines Plans auf einem anderen Gerät identisch ist

In [ ]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

> Hinweis: Derzeit können Sie mit Plan-Klassen nur eine einzige Methode verwenden und müssen sie "forward" nennen.

### Automatische Erstellung von Plänen, die Funktionen sind

Für Funktionen (`@` `sy.func2plan`) können wir den Plan automatisch erstellen, ohne explizit `build` aufrufen zu müssen. Tatsächlich ist der Plan im Moment der Erstellung bereits erstellt.

Um diese Funktionalität zu erhalten, müssen Sie beim Erstellen eines Plans nur ein Argument für den Dekorateur namens `args_shape` festlegen, das eine Liste mit den Formen der einzelnen Argumente sein sollte.

In [ ]:
@sy.func2plan(args_shape=[(-1, 1)])
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

plan_double_abs.is_built

Der Parameter `args_shape` wird intern verwendet, um Scheintensoren mit der angegebenen Form zu erstellen, die zum Erstellen des Plans verwendet werden.

In [ ]:
@sy.func2plan(args_shape=[(1, 2), (-1, 2)])
def plan_sum_abs(x, y):
    s = x + y
    return torch.abs(s)

plan_sum_abs.is_built

Sie können Funktionen auch Statuselemente bereitstellen!

In [ ]:
@sy.func2plan(args_shape=[(1,)], state=(torch.tensor([1]), ))
def plan_abs(x, state):
    bias, = state.read()
    x = x.abs()
    return x + bias

In [ ]:
pointer_plan = plan_abs.send(device_1)
x_ptr = torch.tensor([-1, 0]).send(device_1)
p = pointer_plan(x_ptr)
p.get()

Um mehr darüber zu erfahren, erfahren Sie in Tutorial Teil 8 bis, wie wir Pläne mit Protokollen verwenden!

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notizbuch-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)